In [1]:
import pandas as pd
import numpy as np
import pydantic as py
import json
import re
import random
from nltk.stem.porter import *
import numpy as np
stemmer=PorterStemmer()

In [2]:
# f = open('full-ontology.json')
f = open('new_ontology.json')

data = json.load(f)

In [16]:
df=pd.read_csv('salt_full.csv')


In [17]:
df['foodDescription'] = df['foodDescription'].replace(r'\(\)', '', regex=True) 
df['foodDescription'] = df['foodDescription'].replace(r'[^a-zA-Z, ]', ' ', regex=True) 
df['foodDescription'] = df['foodDescription'].replace(r'andor', ',', regex=True) 
df['foodDescription'] = df['foodDescription'].replace(r'--', ',', regex=True) 
df = df[~df['foodDescription'].str.contains('as ingredient',case=False)]
df['foodDescription'] = df['foodDescription'].str.replace(r', nfs', '', regex=True,case=False) 

In [19]:
df[df['fdcId']==167884]

,fdcId,identifier,identifierType,foodType,foodDescription,value,foodWeight,valueUnit,foodMeasure,measureValue
45775,167884,45776,NDB #,SR Legacy,"Pork, fresh, loin, blade chops , bone in, sep...",85.0,215.0,g,1 chop,183.0
45776,167884,45777,NDB #,SR Legacy,"Pork, fresh, loin, blade chops , bone in, sep...",85.0,85.0,g,3 oz,72.2


In [6]:
df.head()

,fdcId,identifier,identifierType,foodType,foodDescription,value,foodWeight,valueUnit,foodMeasure,measureValue
0,2341725,1,NDB #,Foundation,"Lettuce, romaine, green, raw",0.0,100.0,g,100 g,0.0
1,2341727,2,NDB #,Foundation,"Lettuce, leaf, green, raw",0.0,100.0,g,100 g,0.0
2,2341726,3,NDB #,Foundation,"Lettuce, leaf, red, raw",0.0,100.0,g,100 g,0.0
3,2341724,4,NDB #,Foundation,"Lettuce, iceberg, raw",0.0,100.0,g,100 g,0.0
4,2340760,5,NDB #,Foundation,"Nuts, pine nuts, raw",0.0,100.0,g,100 g,0.0


In [20]:
mask = df['foodDescription'].str.contains('with added',case=False)


In [ ]:
pd.set_option('display.max_colwidth', None)


In [ ]:
df['foodDescription'][mask]

In [21]:
corpus=[]
for description in df['foodDescription']:
  ingredients=description.split(',')
  corpus.append(['_'.join(i.lower().split())  for i in ingredients])
df['corpus']=corpus

In [22]:
type=[]
food=[]
for foodDescription in corpus:
    food.append(foodDescription[0])
    for key in list(data.keys()):
        for ingredient in foodDescription[1:]:
            if ingredient not in data[key]:
                type.append(ingredient)


In [ ]:
data['food']=food
data['type']=type


In [102]:
data.update((x, [i.lower() for i in y]) for x, y in data.items()) #lowering the values
data.update((x, [stemmer.stem(i) for i in y]) for x, y in data.items()) #lowering the values
data.update((x, list(set(data[x]))) for x, y in data.items()) # get unique values


In [ ]:
with open("new_ontology.json", "w") as outfile:
    json.dump(data,outfile)

In [103]:
keys=list(data.keys())
for key in keys:
    print("length of keys",key,len(data[key]))   

length of keys animal 9
length of keys cook 2392
length of keys cuisine 28
length of keys part 77
length of keys texture 7
length of keys color 5
length of keys flavor 7
length of keys ingredient 338
length of keys nutrient 10
length of keys food 2760
length of keys type 4772


In [23]:
dflist=[]
df=df.reset_index(drop=True)
for i in range(len(df)):
  df1={}
  df2={}
  df3={}
  df1['value']=str(np.format_float_positional(df.loc[i]['value'], 3, trim='-'))
  df1['foodDescription']=df.loc[i]['foodDescription']
  df1['saltmetric']="mg"
  df1['foodWeight']="100"
  df1['metric']="g"
  df1['defaultValue']=str(np.format_float_positional(df.loc[i]['value'], 3, trim='-'))
  dflist.append(df1)
  df2['value']=str(np.format_float_positional(df.loc[i]['measureValue'], 3, trim='-'))
  df2['foodDescription']=df.loc[i]['foodDescription']
  # df2['foodMeasure']=df.loc[i]['foodMeasure']
  df2['saltmetric']="mg"
  df2['foodWeight']=str(np.format_float_positional(df.loc[i]['foodWeight'], 3, trim='-'))
  df2['metric']=str(df.loc[i]['valueUnit'])
  df2['defaultValue']=str(np.format_float_positional(df.loc[i]['value'], 3, trim='-'))

  dflist.append(df2)
  isdigit=any(word.isdigit() for word in df.loc[i]['foodMeasure'])
  if isdigit:
    values=re.split(r"[\s\(\)]",df.loc[i]['foodMeasure'])
    df3['value']=str(np.format_float_positional(df.loc[i]['measureValue'], 3, trim='-'))
    df3['foodDescription']=df.loc[i]['foodDescription']
    # df2['foodMeasure']=df.loc[i]['foodMeasure']
    weight=df.loc[i]['foodMeasure']
    df3['saltmetric']="mg"
    df3['foodWeight']=str(values[0])
    df3['metric']=str(values[1])
    df3['defaultValue']=str(np.format_float_positional(df.loc[i]['value'], 3, trim='-'))
    dflist.append(df3)

In [24]:
saltDf=pd.DataFrame(dflist)
saltDf=saltDf.drop_duplicates()
saltDf=saltDf.reset_index(drop=True)

In [131]:
saltDf

,value,foodDescription,saltmetric,foodWeight,metric,defaultValue
0,0,"Lettuce, romaine, green, raw",mg,100,g,0
1,0,"Lettuce, leaf, green, raw",mg,100,g,0
2,0,"Lettuce, leaf, red, raw",mg,100,g,0
3,0,"Lettuce, iceberg, raw",mg,100,g,0
4,0,"Nuts, pine nuts, raw",mg,100,g,0
...,...,...,...,...,...,...
93430,471,"Pie Crust, Cookie type, Graham Cracker, Ready Crust",mg,100,g,471
93431,134,"Pie Crust, Cookie type, Graham Cracker, Ready Crust",mg,28.35,g,471
93432,134,"Pie Crust, Cookie type, Graham Cracker, Ready Crust",mg,1,oz,471
93433,862,"Pie Crust, Cookie type, Graham Cracker, Ready Crust",mg,183,g,471


In [132]:
corpus=[]
for description in saltDf['foodDescription']:
  ingredients=description.split(',')
  corpus.append(['_'.join(i.lower().split())  for i in ingredients])
saltDf['corpus']=corpus

In [133]:
triplets=[]
flag=0
for row in saltDf.itertuples():
    triplet=[]
    for ingredient in row.corpus:
        flag=0
        if row.corpus.index(ingredient)==0:
            relation='food'
            food=ingredient
            subject=row.Index
        else:            
            for key in ['part','cuisine','texture','color','animal','flavor','cook','nutrient','type','ingredient']:
                if stemmer.stem(ingredient) in data[key] and flag==0:
                    relation=key
                    if key not in ['cuisine','cook']:
                        subject=food
                    else:
                        subject=row.Index
                    flag=1
        object=ingredient
        triplet.append([subject,relation,object])
    triplets.append(triplet)



In [135]:
saltDf['triplets']=triplets

In [119]:
visDF=pd.DataFrame(saltDf['triplets'],columns=['subject','edge','object'])

In [136]:
# for triplet in triplets:
#     for i in triplet:
#         visDF=visDF.append({'subject':i[0],'edge':i[1],'object':i[2]},ignore_index=True)


In [122]:
visDF=visDF.drop_duplicates()
visDF=visDF.reset_index(drop=True)

In [123]:
len(visDF)

52011

In [137]:
for key in keys:
    saltDf[key]=saltDf.apply(lambda x: [], axis=1)

In [138]:
for index,triplets in enumerate(saltDf['triplets']):
    for triplet in triplets:
        h,r,o=triplet
        saltDf.loc[index,r].append(o)
    saltDf.loc[index,'nutrient'].append('salt')

In [27]:
saltDf.head()
saltDf.to_excel('dataset.xlsx')

In [26]:
saltDf.head()

,value,foodDescription,saltmetric,foodWeight,metric,defaultValue
0,0,"Lettuce, romaine, green, raw",mg,100,g,0
1,0,"Lettuce, leaf, green, raw",mg,100,g,0
2,0,"Lettuce, leaf, red, raw",mg,100,g,0
3,0,"Lettuce, iceberg, raw",mg,100,g,0
4,0,"Nuts, pine nuts, raw",mg,100,g,0


In [124]:
visDF.to_csv('visualisation.csv')

KeyError: 202

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G=nx.from_pandas_edgelist(visDF[:200],"subject","object",edge_attr='edge',create_using=nx.MultiDiGraph(),edge_key=None)
plt.figure(figsize=(6,6))
pos=nx.spring_layout(G)
# nx.draw(G,with_labels=True,node_color='skyblue',edge_map=plt.cm.Blues,pos=pos)
nx.draw(G,with_labels=True)
nx.draw_networkx_edge_labels(G,pos)
# nx.draw(G,with_labels=True)
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G=nx.from_pandas_dataframe(visDF[:10],"subject","object",['relation'])
plt.figure(figsize=(6,6))
pos=nx.spring_layout(G)
# nx.draw(G,with_labels=True,node_color='skyblue',edge_map=plt.cm.Blues,pos=pos)
nx.draw(G,with_labels=True)
# nx.draw_networkx_edge_labels(G,pos,edge_labels=visDF[:10]['edge'])
# nx.draw(G,with_labels=True)
plt.show()

In [ ]:
visDF[:10]

In [34]:
json_file=[]
for index,row in saltDf.iterrows():
    index,data

In [37]:
saltDf = saltDf.drop(columns='defaultValue')

In [45]:
saltDf.head()

,value,foodDescription,saltmetric,foodWeight,metric
0,0,"Lettuce, romaine, green, raw",mg,100,g
1,0,"Lettuce, leaf, green, raw",mg,100,g
2,0,"Lettuce, leaf, red, raw",mg,100,g
3,0,"Lettuce, iceberg, raw",mg,100,g
4,0,"Nuts, pine nuts, raw",mg,100,g


In [47]:
saltDf = saltDf.rename(columns={'foodDescription': 'fooddescription','foodWeight': 'foodweight'})

In [48]:
json_data = eval(saltDf.to_json(orient='records'))

# Print the JSON data
# print(json_data)

In [49]:
with open("food_db_processed.json", "w") as output_file:
    json.dump(json_data, output_file, indent=4)

In [50]:
#food db
import json
with open("food_db_processed.json","r") as input_file:
    data = json.load(input_file)
unique_entries = set()
filtered_data = []
for entry in data:
    key = (entry["fooddescription"], entry["metric"], entry["foodweight"])
    if key not in unique_entries:
        unique_entries.add(key)
        filtered_data.append(entry)

# Write the filtered data to a new JSON file
with open("food_db_processed.json", "w") as output_file:
    json.dump(filtered_data, output_file, indent=4)

In [ ]:
koperk[tk[]]